# Main ...

 
``config, load, preprocess, train, eval  for  Tick tick bloom``

**Yo DON'T rerun this unless you want to overwrite past models, always fork and do your stuff and 
DON'T forget to change the name**

**``Mission: NNs on expanding avg of severity metadata``**

- wondering how nns perform on metadata!

# Load imports and dependencies

In [1]:
! pip install wandb --upgrade -q

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
allennlp 2.10.1 requires wandb<0.13.0,>=0.10.0, but you have wandb 0.13.8 which is incompatible.


In [2]:
import warnings
import sys
import os
import time
import joblib
import random
from tqdm import tqdm

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import cv2

from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import OrdinalEncoder

import tensorflow as tf
from tensorflow.keras import layers, activations, losses, metrics, models, optimizers, callbacks
from category_encoders.target_encoder import TargetEncoder

warnings.filterwarnings('ignore')

In [3]:
# local utilities imports
from tick_tick_bloom_utils import comp_metric, den2sev_map

In [4]:
# wandb stuff for tracking
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
wandb_login = user_secrets.get_secret("wandb_bloom_tracker")

import wandb
wandb.login(key=wandb_login)

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

# Config

In [5]:
# dot dictionary
class dotdict(dict):
    """dot.notation access to dictionary attributes"""
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__


# Config
config = {}
config = dotdict(config)
config['RANDOM_SEED'] = 18952


config['unique_id'] = int(time.time())
print(f'unique_id: {config.unique_id}')
config['name'] = f'd256_512_256_128_clf_mtdata-{config.unique_id}'

config['PROJECT_NAME'] = 'tick-tick-bloom'
# config['DATA_DIR'] = '../data/'
# config['MODEL_DIR'] = '../models/'
config['SAVE_MODEL'] = True


# # Img config
# config['IMG_SIZE'] = (136, 136)
# config['CHANNELS'] = 3

# training configuration
config['train'] =  dotdict({
                        'epochs': 1000,
                        'batch_size': 256,
                        'validation_split': 0.2,
                        'shuffle': True,
                        'verbose': 1,
                        'lr' : 1e-5
                        })

config['desc'] = """
xp3-to see how nns perform only on metadata(use data upto the point of availibility.)
rmse vs log-loss which is better?? or both could be worst than gbs!
""" 

unique_id: 1673533637


In [6]:
# seed everything
def seed_everything(seed=config.RANDOM_SEED):
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
#     os.environ['TF_CUDNN_DETERMINISTIC'] = '1' 
#     os.environ['TF_DETERMINISTIC_OPS'] = '1'
    os.environ['PYTHONHASHSEED'] = str(seed)

# tf.keras.utils.set_random_seed(config.RANDOM_SEED)  # supposedly sets seed for python, numpy, tf

seed_everything()

# Utils

In [7]:
def keras_rmse_clf(y_true, y_pred):
    """
    valid competetion metric for clf type settings.
    Can be trusted!
    y_true and y_pred should be [0-4]
    """
    y_pred = tf.argmax(y_pred, axis=1)
    y_pred = tf.cast(y_pred, tf.float16)
    y_true = tf.cast(y_true, tf.float16)
    squared_difference = tf.square(y_true - y_pred)
    return tf.sqrt(tf.reduce_mean(squared_difference, axis=-1))

def keras_rmse_reg(y_true, y_pred):
    """
    valid competetion metric for reg type settings.
    Can be trusted!
    y_true and y_pred should be [0-4]
    """
    y_pred = tf.math.round(y_pred)
    y_pred = tf.cast(y_pred, tf.float16)
    y_true = tf.cast(y_true, tf.float16)
    squared_difference = tf.square(y_true - y_pred)
    return tf.sqrt(tf.reduce_mean(squared_difference, axis=-1))


def rmse_loss(y_true, y_pred):
    """loss func to use in reg type settings"""
    return tf.sqrt(losses.mean_squared_error(y_true, y_pred))

# Load data

In [8]:
INPUT_METADATA_DIR = '/kaggle/input/ticktickbloomdataset'

metadata = pd.read_csv(os.path.join(INPUT_METADATA_DIR, 'metadata.csv'))
sub_format = pd.read_csv(os.path.join(INPUT_METADATA_DIR, 'submission_format.csv'))
train_labels = pd.read_csv(os.path.join(INPUT_METADATA_DIR, 'train_labels.csv'))

In [9]:
# IMG_DIR = '/kaggle/input/pull-landsat-data-v1-500m/landsat8_500m_v1'   # landsat 8 data with raw 
# img_files = os.listdir(IMG_DIR)
# img_file_names = [f.split('.')[0] for f in img_files]

# # get only data for those 1k imgs
# metadata_subset = metadata[metadata['uid'].isin(img_file_names)]
# data = metadata_subset[metadata_subset.split == 'train']
# data = data.merge(train_labels, on='uid')

# test_data = metadata[metadata.split == 'train']

In [10]:
# def get_imgs(uids) :
#     imgs = []
#     for uid in uids:
#         img_arr = np.load(IMG_DIR + f'/{uid}.npy')
#         img_arr = np.transpose(img_arr, (2, 1, 0))
#         # resize img
#         img_arr = cv2.resize(img_arr, config.IMG_SIZE)
#         img_arr = img_arr / 255   # normalizeee bro... other wise it's blowing up the networks...
#         imgs.append(img_arr)
#     return np.array(imgs) 


# def get_np_data(split : float = 0.2, task='train'):
#     """Return np data for training and validation."""
#     if task == 'train':
#         print("Loading train and validation data...")
#         x_train_uids, x_val_uids, y_train, y_val = train_val_split(
#             data['uid'],
#             data.severity,
#             val_size=split,
#             random_state=config.RANDOM_SEED,
#             stratify=data.severity
#         )

#         x_train = get_imgs(x_train_uids)
#         x_val = get_imgs(x_val_uids)

#         return x_train, y_train, x_val, y_val


#     if task == 'test':
#         test_ids = test_data.uids
#         x_test
#         return x_test

In [11]:
# x_train, y_train, x_val, y_val = get_np_data()
# print(y_train.value_counts(normalize=True))
# print(y_val.value_counts(normalize=True))
# print('Done')

In [12]:
#  get data
metadata.date = pd.to_datetime(metadata.date)

region = pd.concat((train_labels, sub_format[['region', 'uid']]), axis=0)

data = pd.merge(metadata, region, on='uid', how='left')
print(data.shape)

(23570, 8)


In [13]:
# seasons
seasons = {
    1: 1,
    2: 1,
    3: 2,
    4: 2,
    5: 2,
    6: 3,
    7: 3,
    8: 3,
    9: 4,
    10: 4,
    11: 4,
    12: 1
}


#  most of the samples are collected in the months of June, July, August.

# add date time fts.
data['month'] = data.date.dt.month
data['year'] = data.date.dt.year
data['week'] = data.date.dt.isocalendar().week
# data['day_of_year'] = data.date.dt.
data['season'] = data.month.map(seasons)

In [14]:
data.sort_values(by='date', inplace=True)

In [15]:
data['expanding_severity'] = data.severity.expanding().mean()
data['expanding_severity'] = data['expanding_severity'].apply(np.round)
data

,uid,latitude,longitude,date,split,region,severity,density,month,year,week,season,expanding_severity
4387,evep,44.847993,-93.476318,2013-01-04,train,midwest,1.0,115.0,1,2013,1,1,1.0
13644,paev,44.822478,-93.367962,2013-01-04,train,midwest,1.0,1884.0,1,2013,1,1,1.0
5566,gdxr,44.877646,-93.557842,2013-01-04,train,midwest,1.0,1416.0,1,2013,1,1,1.0
6144,guny,44.878889,-93.490833,2013-01-04,train,midwest,1.0,558.0,1,2013,1,1,1.0
5317,fwbt,44.850500,-93.515700,2013-01-04,train,midwest,1.0,476.0,1,2013,1,1,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
12443,nsoi,36.736800,-121.734000,2021-12-29,test,west,NaN,NaN,12,2021,52,1,2.0
17559,thki,36.725400,-121.730000,2021-12-29,test,west,NaN,NaN,12,2021,52,1,2.0
17452,teuu,36.772300,-121.788000,2021-12-29,test,west,NaN,NaN,12,2021,52,1,2.0
14254,prfi,36.751800,-121.742000,2021-12-29,test,west,NaN,NaN,12,2021,52,1,2.0


In [16]:
train_data = data[data.split == 'train']
test_data = data[data.split == 'test']

In [17]:
# expanding avg of severity
mse(train_data.severity, train_data.expanding_severity, squared=False)

1.2085810811762927

In [18]:
grp_by_region = data.groupby('region').severity.expanding(1).mean()
grp_by_region = grp_by_region.map(np.round)

grp_by_region['west'].fillna(2, inplace=True)
grp_by_region['northeast'].fillna(2, inplace=True)
print(grp_by_region.isna().sum())   # 5 --> 0.89416

print(mse(train_data.severity.sort_index(), grp_by_region.droplevel(0).loc[train_data.index].sort_index(), squared=False))

data['expndng_sev_by_reg'] = np.nan

south = data.region == 'south'
midwest = data.region == 'midwest'
northeast = data.region == 'northeast'
west = data.region == 'west'

data.loc[south , 'expndng_sev_by_reg'] = grp_by_region['south']
data.loc[midwest , 'expndng_sev_by_reg'] = grp_by_region['midwest']
data.loc[northeast , 'expndng_sev_by_reg'] = grp_by_region['northeast']
data.loc[west , 'expndng_sev_by_reg'] = grp_by_region['west']

print(data.shape)
data.isna().sum()

data.sort_index()

0
0.894165010958815
(23570, 14)


,uid,latitude,longitude,date,split,region,severity,density,month,year,week,season,expanding_severity,expndng_sev_by_reg
0,aabm,39.080319,-86.430867,2018-05-14,train,midwest,1.0,585.0,5,2018,20,2,2.0,2.0
1,aabn,36.559700,-121.510000,2016-08-31,test,west,NaN,NaN,8,2016,35,3,2.0,4.0
2,aacd,35.875083,-78.878434,2020-11-19,train,south,1.0,290.0,11,2020,47,4,2.0,2.0
3,aaee,35.487000,-79.062133,2016-08-24,train,south,1.0,1614.0,8,2016,34,3,2.0,2.0
4,aaff,38.049471,-99.827001,2019-07-23,train,midwest,3.0,111825.0,7,2019,30,3,2.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23565,zzvv,36.708500,-121.749000,2014-12-02,test,west,NaN,NaN,12,2014,49,1,2.0,4.0
23566,zzwo,39.792190,-99.971050,2017-06-19,train,midwest,2.0,48510.0,6,2017,25,3,2.0,2.0
23567,zzwq,35.794000,-79.012551,2015-03-24,train,south,1.0,1271.0,3,2015,13,2,2.0,1.0
23568,zzyb,35.742000,-79.238600,2016-11-21,train,south,1.0,9682.0,11,2016,47,4,2.0,2.0


In [19]:
# # MY ASSUMPTION: less the missing values --> more inital samples in the group are not test --> less imputations/ffills needed --> much realiable score!


In [20]:
grp_by_rs = data.groupby(['region', 'season']).severity.expanding(1).mean()
grp_by_rs = grp_by_rs.map(np.round)
print(grp_by_rs.isna().sum()) # 5 --> .86

data['expanding_sev_rs'] =  grp_by_rs.droplevel(0).droplevel(0).sort_index()
# fillna with expanding sev by region
data['expanding_sev_rs'] = np.where(data.expanding_sev_rs.isna(), data.expndng_sev_by_reg, data.expanding_sev_rs)

print(mse(train_data.severity.sort_index(), data['expanding_sev_rs'].sort_index()[data.split == 'train'], squared=False))

# #  make submission for expanding severity by region and season

# expanding_sev_rs = data[data.split == 'test'][['uid', 'expanding_sev_rs']]          # picking up only uids and expanding_sev_rs from test samples
# expanding_sev_rs.expanding_sev_rs = expanding_sev_rs.expanding_sev_rs.astype(int)   # casting to int
# expanding_sev_rs.sort_values(by='uid', inplace=True)                                # sorting by uid -- safest option
# expanding_sev_rs.reset_index(drop=True, inplace=True)                               # matching indexes with submissoin

# sub_format.severity = expanding_sev_rs.expanding_sev_rs
# sub_format.severity.value_counts()  # expected 0.8594349134502333

# sub_format.to_csv('expanding_sev_rs_preds.csv', index=False)

5
0.8594349134502333


In [21]:
train_labels.severity.value_counts(normalize=True)
# since test and train dists are almost similar my ideal model should follow this dist!

1    0.439449
4    0.207913
2    0.189859
3    0.159379
5    0.003400
Name: severity, dtype: float64

In [22]:
data.head()

,uid,latitude,longitude,date,split,region,severity,density,month,year,week,season,expanding_severity,expndng_sev_by_reg,expanding_sev_rs
4387,evep,44.847993,-93.476318,2013-01-04,train,midwest,1.0,115.0,1,2013,1,1,1.0,1.0,1.0
13644,paev,44.822478,-93.367962,2013-01-04,train,midwest,1.0,1884.0,1,2013,1,1,1.0,1.0,1.0
5566,gdxr,44.877646,-93.557842,2013-01-04,train,midwest,1.0,1416.0,1,2013,1,1,1.0,1.0,1.0
6144,guny,44.878889,-93.490833,2013-01-04,train,midwest,1.0,558.0,1,2013,1,1,1.0,1.0,1.0
5317,fwbt,44.850500,-93.515700,2013-01-04,train,midwest,1.0,476.0,1,2013,1,1,1.0,1.0,1.0


In [23]:
data.isna().sum()

uid                      0
latitude                 0
longitude                0
date                     0
split                    0
region                   0
severity              6510
density               6510
month                    0
year                     0
week                     0
season                   0
expanding_severity       0
expndng_sev_by_reg       0
expanding_sev_rs         0
dtype: int64

In [24]:
all_train = data[data.split == 'train']
all_train.sort_values(by='uid', inplace=True)
all_train.reset_index(drop=True, inplace=True)
all_train.shape

(17060, 15)

In [25]:
all_train.head()

,uid,latitude,longitude,date,split,region,severity,density,month,year,week,season,expanding_severity,expndng_sev_by_reg,expanding_sev_rs
0,aabm,39.080319,-86.430867,2018-05-14,train,midwest,1.0,585.0,5,2018,20,2,2.0,2.0,1.0
1,aacd,35.875083,-78.878434,2020-11-19,train,south,1.0,290.0,11,2020,47,4,2.0,2.0,2.0
2,aaee,35.487000,-79.062133,2016-08-24,train,south,1.0,1614.0,8,2016,34,3,2.0,2.0,2.0
3,aaff,38.049471,-99.827001,2019-07-23,train,midwest,3.0,111825.0,7,2019,30,3,2.0,2.0,2.0
4,aafl,39.474744,-86.898353,2021-08-23,train,midwest,4.0,2017313.0,8,2021,34,3,2.0,2.0,2.0


In [26]:
test_data = metadata[metadata.split == 'test']
test_data = test_data.merge(sub_format, on='uid')

test_data['month'] = test_data.date.dt.month
test_data['year'] = test_data.date.dt.year
test_data['season'] = test_data.month.map(seasons)

In [27]:
all_train.columns

Index(['uid', 'latitude', 'longitude', 'date', 'split', 'region', 'severity',
       'density', 'month', 'year', 'week', 'season', 'expanding_severity',
       'expndng_sev_by_reg', 'expanding_sev_rs'],
      dtype='object')

In [28]:
req_cols = ['region', 'month', 'year', 'season', 'expanding_sev_rs']

In [29]:
X_ , y_ = all_train[req_cols], all_train['severity']
X_.shape, y_.shape

((17060, 5), (17060,))

In [30]:
X_.isna().sum().sum(), y_.isna().sum()

(0, 0)

In [31]:
X_train_,X_val_, y_train_, y_val_ = train_test_split(X_, y_, test_size=0.20, random_state=config.RANDOM_SEED, stratify=y_)
X_train_.shape, y_train_.shape, X_val_.shape, y_val_.shape

((13648, 5), (13648,), (3412, 5), (3412,))

In [32]:
test_data['expanding_severity'] = data[data.split == 'test']['expanding_severity'].sort_index().values
test_data['expndng_sev_by_reg'] = data[data.split == 'test']['expndng_sev_by_reg'].sort_index().values
test_data['expanding_sev_rs'] = data[data.split == 'test']['expanding_sev_rs'].sort_index().values

In [33]:
X_test_ = test_data[req_cols]
X_test_.shape

(6510, 5)

# Preprocess

In [34]:
# # change labels to 0-3(model works this way) instead of 1-4 given range(given severity)
# -1 for to make labels look like sparse encoded labels

y_train = y_train_  -1
y_val = y_val_ - 1

y_train.value_counts(normalize=True), y_val.value_counts(normalize=True)  # guessing alwyas 0 gives 43% acc

(0.0    0.439478
 3.0    0.207943
 1.0    0.189845
 2.0    0.159364
 4.0    0.003370
 Name: severity, dtype: float64,
 0.0    0.439332
 3.0    0.207796
 1.0    0.189918
 2.0    0.159437
 4.0    0.003517
 Name: severity, dtype: float64)

In [35]:
y = y_ - 1
y.value_counts()

0.0    7497
3.0    3547
1.0    3239
2.0    2719
4.0      58
Name: severity, dtype: int64

In [36]:
# #  target encode the cat fts.

# te = TargetEncoder(cols=['region', 'month', 'year', 'season'])
# te.fit(X_train_, y_train)
# X_train =  te.transform(X_train_)
# X_val = te.transform(X_val_)

# X_test = te.transform(X_test_)
# X_test

In [37]:
X_train_.dtypes

region               object
month                 int64
year                  int64
season                int64
expanding_sev_rs    float64
dtype: object

In [38]:
#  Encode region
# from category_encoders.ordinal import OrdinalEncoder as COE

oe = OrdinalEncoder()
X_train = oe.fit_transform(X_train_)
X_test = oe.transform(X_test_)
X_val = oe.transform(X_val_)

In [39]:
X_train = pd.DataFrame(X_train, columns=X_train_.columns, index=X_train_.index)
X_test = pd.DataFrame(X_test, columns=X_test_.columns, index=X_test_.index)
X_val = pd.DataFrame(X_val, columns=X_val_.columns, index=X_val_.index)

In [40]:
# Normalize values




# Model

In [41]:
def get_model(mdtype='reg'):
    print(f'Loading {mdtype} type model...')
    #     input_shape = (*config.IMG_SIZE, config.CHANNELS)

    if mdtype == 'clf':
        loss = losses.SparseCategoricalCrossentropy()
        comp_metric = keras_rmse_clf
        last_layer = layers.Dense(5, activation='softmax')

    if mdtype == 'reg':
        loss = rmse_loss
        comp_metric = keras_rmse_reg
        last_layer = layers.Dense(1)
    
    
    
    input_imgs = layers.Input(shape=(X_train.shape[1],))
    #     x = layers.Conv2D(32, (3, 3), activation='relu')(input_imgs)
    #     x = layers.MaxPooling2D((2, 2))(x)
    #     x = layers.Flatten()(x)
    x = layers.Dense(256, activation='relu')(input_imgs)
    x = layers.Dense(512, activation='relu')(x)
    x = layers.Dense(256, activation='relu')(x)
    x = layers.Dense(128, activation='relu')(x)
    output = last_layer(x)

    model = models.Model(inputs=input_imgs, outputs=output, name=config.name)
    
    

    model.compile(optimizer=optimizers.Adam(learning_rate=config.train.lr),
                    loss = loss,
                    metrics=[
                        comp_metric,
                        metrics.SparseCategoricalAccuracy(name='acc')
                    ])

    return model

In [42]:
model = get_model('clf')
model.summary()

Loading clf type model...


2023-01-12 14:27:19.450274: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-12 14:27:19.451299: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-12 14:27:19.584702: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-12 14:27:19.585802: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-12 14:27:19.586826: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from S

Model: "d256_512_256_128_clf_mtdata-1673533637"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 5)]               0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               1536      
_________________________________________________________________
dense_2 (Dense)              (None, 512)               131584    
_________________________________________________________________
dense_3 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_4 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense (Dense)                (None, 5)                 645       
Total params: 297,989
Trainable params: 297,989
Non-trainable params: 0
______________________

In [43]:
# model.fit(X_train, y_train)

# Train and eval

In [44]:
def train_(model, config=config, x_train=X_train, y_train=y_train, debug=None):
    """fits given model to x_train and y_train"""
    
    train_config = config['train']
    my_callbacks = []
    
    earlystopping = callbacks.EarlyStopping(patience=15, monitor='val_loss', restore_best_weights=True)
    my_callbacks.append(earlystopping)
    callbacks.ReduceLROnPlateau(
                    monitor="val_acc",
                    factor=0.5,
                    patience=5,
                    verbose=1,
                    mode="auto",
                    min_delta=0.01,
                    cooldown=0,
                    min_lr=0)
    
    try:
        wandb_callback = wandb.keras.WandbCallback(
            monitor='val_loss',
            log_weights=True,
            log_gradients=True,
            save_model=False,
            training_data=(x_train, y_train),
            log_batch_frequency=None,
        )

        my_callbacks.append(wandb_callback)
    except:
        print('wandb not tracking')
        
    print(f'Training model... {config.name}')
    if debug == True:
        epochs = 1000
    else:
        epochs = train_config.epochs
    history = model.fit(
                x_train, y_train,
                epochs=epochs,
                batch_size=train_config.batch_size, 
                callbacks=my_callbacks, 
                validation_split=0.2, 
                shuffle=True, 
                verbose=1 
            )

    return model, history


def eval_(model, x_val=X_val, y_val=y_val):
    print('Evaluating model....')
    model.evaluate(x_val, y_val, return_dict=True)


In [45]:
def train_eval(model=None, mdtype='reg', X_train=X_train, y_train=y_train, X_val=X_val, y_val=y_val, debug=False):
    
    if model is None:
        print('Getting New model')
        model = get_model()
    
    # train
    model, history = train_(model, config, X_train, y_train=y_train, debug=debug)  # try to overfit thsi batch
    # eval
    eval_(model)

    # classification report
    y_pred = model.predict(X_val)
    if mdtype == 'clf':
        y_pred_hard = np.argmax(y_pred, axis=1)             # fuck this shit forgot to 
    if mdtype == 'reg':
        y_pred_hard = np.round(y_pred)
        
    print(y_pred_hard)
    error = mse(y_val, y_pred_hard, squared=False)
    print("Comp Metric: ", error)
    cr = classification_report(y_val, y_pred_hard)     # +1 to account for 0-4 as it should be 1-5 originallly
    print(cr)


In [46]:
model = get_model('clf')
train_eval(model, 'clf', y_train=y_train, y_val=y_val)

Loading clf type model...
wandb not tracking
Training model... d256_512_256_128_clf_mtdata-1673533637
Epoch 1/1000


2023-01-12 14:27:23.208884: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


43/43 [==============================] - 2s 9ms/step - loss: 1.5656 - keras_rmse_clf: 1.3923 - acc: 0.3894 - val_loss: 1.4928 - val_keras_rmse_clf: 1.1817 - val_acc: 0.4414
Epoch 2/1000
43/43 [==============================] - 0s 8ms/step - loss: 1.4460 - keras_rmse_clf: 1.2874 - acc: 0.4525 - val_loss: 1.4006 - val_keras_rmse_clf: 1.3816 - val_acc: 0.4703
Epoch 3/1000
43/43 [==============================] - 0s 6ms/step - loss: 1.3669 - keras_rmse_clf: 1.4610 - acc: 0.4933 - val_loss: 1.3329 - val_keras_rmse_clf: 1.5315 - val_acc: 0.5198
Epoch 4/1000
43/43 [==============================] - 0s 5ms/step - loss: 1.3034 - keras_rmse_clf: 1.5694 - acc: 0.5375 - val_loss: 1.2747 - val_keras_rmse_clf: 1.5999 - val_acc: 0.5491
Epoch 5/1000
43/43 [==============================] - 0s 5ms/step - loss: 1.2474 - keras_rmse_clf: 1.6278 - acc: 0.5663 - val_loss: 1.2233 - val_keras_rmse_clf: 1.6394 - val_acc: 0.5725
Epoch 6/1000
43/43 [==============================] - 0s 5ms/step - loss: 1.1980 - 

In [47]:
config.train.epochs = 1000   # mission: achieve lowest loss possible..
config.desc = "Bigger dense model on mtdata sev by rs log-loss"

In [49]:
! pip install -U setuptools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 8.9 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: setuptools
    Found existing installation: setuptools 59.8.0
    Uninstalling setuptools-59.8.0:
      Successfully uninstalled setuptools-59.8.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dask-cudf 21.10.1 requires cupy-cuda114, which is not installed.
beatrix-jupyterlab 3.1.7 requires google-cloud-bigquery-storage, which is not installed.
nnabla 1.32.0 requires protobuf<=3.19.4; platform_system != "Windows", but you have protobuf 3.20.3 which is incompatible.
gcsfs 2022.5.0 requires fsspec==2022.5.0, but you have fsspec 2022.11.0 which is incompatible.
dask-cudf 21.10.1 requires dask==2021.09.1, but you have dask 2022.2.0 which is incompatible.
dask-cudf 21.10.1 requires distributed==2021.09.1, but you have distributed 2022.2.0 which is incompa

In [54]:
with wandb.init(project=config.PROJECT_NAME, config=config, name=config.name):
    model = get_model('clf')
    train_eval(model, 'clf', y_train=y_train, y_val=y_val)

Problem at: /tmp/ipykernel_23/223850537.py 1 <module>


Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/wandb/sdk/wandb_init.py", line 1108, in init
    run = wi.init()
  File "/opt/conda/lib/python3.7/site-packages/wandb/sdk/wandb_init.py", line 742, in init
    raise error
wandb.errors.CommError: Error communicating with wandb process, exiting...
For more info see: https://docs.wandb.ai/library/init#init-start-error
wandb: ERROR Abnormal program exit


Exception: problem

In [50]:
preds = np.argmax(model.predict(X_val), axis=1)  # 0.9779215704185079
mse(y_val, preds.ravel(), squared=False)

0.9779215704185079

In [51]:
pd.Series(preds).value_counts(normalize=True)

0    0.687280
3    0.221571
2    0.091149
dtype: float64

In [52]:
y_val.value_counts(normalize=True)

0.0    0.439332
3.0    0.207796
1.0    0.189918
2.0    0.159437
4.0    0.003517
Name: severity, dtype: float64

In [ ]:
# history.history

In [ ]:
# model = models.load_model('/kaggle/working/d128_rmse_lndsat8_raw_v1-1673283452.h5', custom_objects={'comp_loss': comp_loss})
# preds = model.predict(X_val)
# int_preds = np.round(preds)
# mse(y_val, int_preds, squared=False)

# Save something..

In [53]:
# save model
if config.SAVE_MODEL:
    model.save(config.name + '.h5')
    print("Model saved as ",config.name + '.h5')

Model saved as  d256_512_256_128_clf_mtdata-1673533637.h5


# Make submission

In [55]:
model.summary()

Model: "d256_512_256_128_clf_mtdata-1673533637"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 5)]               0         
_________________________________________________________________
dense_6 (Dense)              (None, 256)               1536      
_________________________________________________________________
dense_7 (Dense)              (None, 512)               131584    
_________________________________________________________________
dense_8 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_9 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_5 (Dense)              (None, 5)                 645       
Total params: 297,989
Trainable params: 297,989
Non-trainable params: 0
______________________

In [56]:
X_test.shape

(6510, 5)

In [57]:
# test_preds = np.round(model.predict(X_test)).ravel()
test_preds = np.argmax(model.predict(X_test), axis=1)
test_preds = test_preds + 1
sub_format.severity = test_preds
sub_format.severity = sub_format.severity.astype(int) 
sub_format.severity.value_counts()

1    2981
4    2383
3    1145
2       1
Name: severity, dtype: int64

In [59]:
save_file_to = f'{config.name}_preds.csv'
print(f'saving file to {save_file_to}')
sub_format.to_csv(save_file_to, index=False) # expect @ 0.979 0.98

saving file to d256_512_256_128_clf_mtdata-1673533637_preds.csv


# So...

- NNs with log_loss not at all improving mostly coz of loss -func! --> I thought but
- NNs with log loss is better compared to rmse-loss??
- 0.9898369849328295 prev best of 0.97777 is with leaked metadata and a failuree!!

# ToDos:

- GET a bigger network to overfit and train it to max level maybe and see how far it can go??
- **Try to beat expanding avg_severity_by_region with the help of imgs, Other wise no use for img data**